# SCaling and Queues


## reference

https://kubernetes.io/docs/tasks/job/fine-parallel-processing-work-queue/


In this example, we will run a Kubernetes Job with multiple parallel worker processes in a given pod.

In this example, as each pod is created, it picks up one unit of work from a task queue, processes it, and repeats until the end of the queue is reached.


1. Start a storage service to hold the work queue.
2. Create a queue, and fill it with messages. 
3. Start a Job that works on tasks from the queue. 


In [1]:
%env k=microk8s.kubectl

env: k=microk8s.kubectl


In [2]:
! $k get all

NAME                 TYPE        CLUSTER-IP     EXTERNAL-IP   PORT(S)   AGE
service/kubernetes   ClusterIP   10.152.183.1   <none>        443/TCP   23h


In [4]:
! microk8s enable dns 

Enabling DNS
Applying manifest
serviceaccount/coredns created
configmap/coredns created
deployment.apps/coredns created
service/kube-dns created
clusterrole.rbac.authorization.k8s.io/coredns created
clusterrolebinding.rbac.authorization.k8s.io/coredns created
Restarting kubelet
DNS is enabled


In [5]:
! microk8s.status |grep dns

dns: enabled


In [42]:
# ! mkdir -p redis

In [43]:
#! (cd redis; wget https://kubernetes.io/examples/application/job/redis/redis-pod.yaml)

--2020-03-30 21:32:59--  https://kubernetes.io/examples/application/job/redis/redis-pod.yaml
Resolving kubernetes.io (kubernetes.io)... 147.75.40.148
Connecting to kubernetes.io (kubernetes.io)|147.75.40.148|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 240 [application/x-yaml]
Saving to: ‘redis-pod.yaml.1’

redis-pod.yaml.1    100%[===================>]     240  --.-KB/s    in 0s      

2020-03-30 21:33:00 (7.06 MB/s) - ‘redis-pod.yaml.1’ saved [240/240]



In [6]:
! ls -l redis

total 60
-rw-r--r-- 1 ubuntu root  115 Apr  1 18:43 Dockerfile
-rw-r--r-- 1 ubuntu root  115 Apr  1 18:43 Dockerfile.1
-rw-r--r-- 1 ubuntu root  195 Apr  1 18:43 Makefile
-rw-r--r-- 1 ubuntu root  235 Apr  1 18:43 job.yaml
-rw-r--r-- 1 ubuntu root  244 Apr  1 18:43 job.yaml.1
-rw-r--r-- 1 ubuntu root  240 Apr  1 18:43 redis-pod.yaml
-rw-r--r-- 1 ubuntu root  240 Apr  1 18:43 redis-pod.yaml.1
-rw-r--r-- 1 ubuntu root  135 Apr  1 18:43 redis-service.yaml
-rw-r--r-- 1 ubuntu root  135 Apr  1 18:43 redis-service.yaml.1
-rw-r--r-- 1 ubuntu root 5545 Apr  1 18:43 rediswq.py
-rw-r--r-- 1 ubuntu root 5545 Apr  1 18:43 rediswq.py.1
-rw-r--r-- 1 ubuntu root  657 Apr  1 18:43 worker.py
-rw-r--r-- 1 ubuntu root  657 Apr  1 18:43 worker.py.1


In [7]:
'''
%%bash

cd redis
wget https://kubernetes.io/examples/application/job/redis/redis-service.yaml
wget https://kubernetes.io/examples/application/job/redis/Dockerfile
wget https://kubernetes.io/examples/application/job/redis/job.yaml
wget https://kubernetes.io/examples/application/job/redis/rediswq.py
wget https://kubernetes.io/examples/application/job/redis/worker.py
'''

'\n%%bash\n\ncd redis\nwget https://kubernetes.io/examples/application/job/redis/redis-service.yaml\nwget https://kubernetes.io/examples/application/job/redis/Dockerfile\nwget https://kubernetes.io/examples/application/job/redis/job.yaml\nwget https://kubernetes.io/examples/application/job/redis/rediswq.py\nwget https://kubernetes.io/examples/application/job/redis/worker.py\n'

In [8]:
! ls -l redis

total 60
-rw-r--r-- 1 ubuntu root  115 Apr  1 18:43 Dockerfile
-rw-r--r-- 1 ubuntu root  115 Apr  1 18:43 Dockerfile.1
-rw-r--r-- 1 ubuntu root  195 Apr  1 18:43 Makefile
-rw-r--r-- 1 ubuntu root  235 Apr  1 18:43 job.yaml
-rw-r--r-- 1 ubuntu root  244 Apr  1 18:43 job.yaml.1
-rw-r--r-- 1 ubuntu root  240 Apr  1 18:43 redis-pod.yaml
-rw-r--r-- 1 ubuntu root  240 Apr  1 18:43 redis-pod.yaml.1
-rw-r--r-- 1 ubuntu root  135 Apr  1 18:43 redis-service.yaml
-rw-r--r-- 1 ubuntu root  135 Apr  1 18:43 redis-service.yaml.1
-rw-r--r-- 1 ubuntu root 5545 Apr  1 18:43 rediswq.py
-rw-r--r-- 1 ubuntu root 5545 Apr  1 18:43 rediswq.py.1
-rw-r--r-- 1 ubuntu root  657 Apr  1 18:43 worker.py
-rw-r--r-- 1 ubuntu root  657 Apr  1 18:43 worker.py.1


# Run the redis cli interactively

in a terminal windo do this

## kubectl run -i --tty temp --image redis --command "/bin/sh"

# OOPS - need a basic redis service up

In [9]:
! $k create -f ./redis/redis-pod.yaml

pod/redis-master created


In [10]:
! $k create -f ./redis/redis-service.yaml

service/redis created


# simulate master

1. open new teminal and exec it into master tester

## Filling the Queue with tasks

Now let’s fill the queue with some “tasks”. In our example, our tasks are just strings to be printed.

Start a temporary interactive pod for running the Redis CLI.

```
kubectl run -i --tty temp --image redis --command "/bin/sh"
Waiting for pod default/redis2-c7h78 to be running, status is Pending, pod ready: false
Hit enter for command prompt

redis-cli -h redis

```
rpush job2 "apple"



rpush job2 "cherry"


rpush job2 "date"


rpush job2 "fig"


rpush job2 "grape"


rpush job2 "lemon"


rpush job2 "melon"


rpush job2 "orange"






lrange job2 0 -1

```


Now hit enter, start the redis CLI, and create a list with some work items in it.

# redis-cli -h redis
redis:6379> rpush job2 "apple"
(integer) 1
redis:6379> rpush job2 "banana"
(integer) 2
redis:6379> rpush job2 "cherry"
(integer) 3
redis:6379> rpush job2 "date"
(integer) 4
redis:6379> rpush job2 "fig"
(integer) 5
redis:6379> rpush job2 "grape"
(integer) 6
redis:6379> rpush job2 "lemon"
(integer) 7
redis:6379> rpush job2 "melon"
(integer) 8
redis:6379> rpush job2 "orange"
(integer) 9
redis:6379> lrange job2 0 -1


In [16]:
%env k=microk8s.kubectl

env: k=microk8s.kubectl


In [20]:
! $k delete -f ./redis/job.yaml

job.batch "job-wq-2" deleted


In [21]:
#! $k --help

In [22]:
! $k create -f ./redis/job.yaml

job.batch/job-wq-2 created


In [23]:
! $k get pods

NAME                    READY   STATUS              RESTARTS   AGE
job-wq-2-txhhh          0/1     ContainerCreating   0          3s
job-wq-2-zr24j          0/1     ContainerCreating   0          3s
redis-master            1/1     Running             0          9m4s
temp-5878477f6c-bk99j   1/1     Running             0          2m39s


In [24]:
! $k describe jobs/job-wq-2

Name:           job-wq-2
Namespace:      default
Selector:       controller-uid=b500de9c-d76f-408e-a71c-d1eb37176cf6
Labels:         controller-uid=b500de9c-d76f-408e-a71c-d1eb37176cf6
                job-name=job-wq-2
Annotations:    <none>
Parallelism:    2
Completions:    <unset>
Start Time:     Thu, 02 Apr 2020 18:13:40 +0000
Pods Statuses:  2 Running / 0 Succeeded / 0 Failed
Pod Template:
  Labels:  controller-uid=b500de9c-d76f-408e-a71c-d1eb37176cf6
           job-name=job-wq-2
  Containers:
   c:
    Image:        tbutzer/job-wq-2
    Port:         <none>
    Host Port:    <none>
    Environment:  <none>
    Mounts:       <none>
  Volumes:        <none>
Events:
  Type    Reason            Age   From            Message
  ----    ------            ----  ----            -------
  Normal  SuccessfulCreate  5s    job-controller  Created pod: job-wq-2-zr24j
  Normal  SuccessfulCreate  5s    job-controller  Created pod: job-wq-2-txhhh


In [25]:
#!$k logs pods/job-wq-2-25zcv

In [26]:
#!$k logs pods/job-wq-2-nj7wb

In [27]:
! sleep 15

In [47]:
%%bash

POD=$(microk8s.kubectl get pods  -o name |grep wq-2)
echo $POD
#$k get logs pod ${POD}

for pod in $POD; do echo . ; $k logs $pod; echo ------;  done

pod/job-wq-2-txhhh pod/job-wq-2-zr24j
.
Worker with sessionID: e52935d4-4d0d-4e3b-9702-96846cc46673
Initial queue state: empty=False
Working on apple
Working on apple
Working on apple
Working on apple
Working on apple
Working on apple
Working on apple
Working on apple
Working on apple
Working on apple
Working on apple
Queue empty, exiting
------
.
Worker with sessionID: 106f6849-f2fe-4bd0-b269-39dff186b766
Initial queue state: empty=False
Working on apple
Working on apple
Working on apple
Working on apple
Working on apple
Working on apple
Working on apple
Working on apple
Working on apple
Working on apple
Working on apple
Queue empty, exiting
------


In [44]:
! $k get pods

NAME                    READY   STATUS      RESTARTS   AGE
job-wq-2-txhhh          0/1     Completed   0          117s
job-wq-2-zr24j          0/1     Completed   0          117s
redis-master            1/1     Running     0          10m
temp-5878477f6c-bk99j   1/1     Running     0          4m33s


In [46]:
! $k logs pods/job-wq-2-txhhh


Worker with sessionID: e52935d4-4d0d-4e3b-9702-96846cc46673
Initial queue state: empty=False
Working on apple
Working on apple
Working on apple
Working on apple
Working on apple
Working on apple
Working on apple
Working on apple
Working on apple
Working on apple
Working on apple
Queue empty, exiting
